### 토큰화

### 파일 로드

In [1]:
import pandas as pd
import os

base_dir = os.getcwd() + "/"

base_dir

#base_dir = "/content/drive/MyDrive/bookend/dev/text-style-classify/basic/"

'c:\\Dev\\Github\\text-style-classify\\basic/'

In [2]:
def split_dataframe(df, n):
    split_dfs = []
    chunk_size = len(df) // n
    for i in range(n):
        start_index = i * chunk_size
        if i == n - 1:  # 마지막 부분은 나머지를 포함
            end_index = len(df)
        else:
            end_index = (i + 1) * chunk_size
        split_dfs.append(df.iloc[start_index:end_index].copy())
    return split_dfs

dfs = split_dataframe(pd.read_csv(base_dir + "datasets/written.csv"), 10)

In [3]:
for i in range(len(dfs)):
    dfs[i].to_csv(base_dir + "datasets/written_" + str(i) + ".csv", index=False)

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')"""

In [ ]:
"""df1 = pd.read_csv(base_dir + "datasets/translate.csv")
df2 = pd.read_csv(base_dir + "datasets/written_0.csv")
df3 = pd.read_csv(base_dir + "datasets/written_1.csv")
df4 = pd.read_csv(base_dir + "datasets/written_2.csv")
df5 = pd.read_csv(base_dir + "datasets/written_3.csv")"""

In [ ]:
# df2 = pd.read_csv(base_dir + "datasets/written.csv")

df2s = []

for i in range(10):
    print(f"Reading {i}")
    df2s.append(pd.read_pickle(base_dir + "datasets/written_" + str(i) + ".pkl"))
df2 = pd.concat(df2s)

df2 = df2[['author', 'sentence', 'tiktoken_encodes']]

df2.to_pickle(base_dir + "datasets/written_t1.pkl")

In [ ]:
df3s = []

for i in range(3):
    print(f"Reading {i+3}")
    df3s.append(pd.read_pickle(base_dir + "datasets/written_" + str(i+3) + ".pkl"))
df3 = pd.concat(df3s)

df3 = df3[['author', 'sentence', 'tiktoken_encodes']]

df3.to_pickle(base_dir + "datasets/written_t2.pkl")

In [ ]:
df4s = []

for i in range(4):
    print(f"Reading {i+6}")
    df4s.append(pd.read_pickle(base_dir + "datasets/written_" + str(i+6) + ".pkl"))
df4 = pd.concat(df4s)

df4 = df4[['author', 'sentence', 'tiktoken_encodes']]

df4.to_pickle(base_dir + "datasets/written_t3.pkl")

### BERT 토크나이저

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def bert_tokenizer(sentence):
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=64,
        padding="max_length",
        return_attention_mask=False,
        return_tensors='pt',
        truncation=True
    )
    tokens = encoding['input_ids'][0]
    token_list = tokenizer.convert_ids_to_tokens(tokens)
    return pd.Series([tokens, token_list])

In [ ]:
#df1[["bert_encodes", "bert_tokens"]] = df1["sentence"].apply(bert_tokenizer)

In [ ]:
"""df2[["bert_encodes", "bert_tokens"]] = df2["sentence"].apply(bert_tokenizer)"""

### Mecab 토크나이저

In [ ]:
from mecab import MeCab

mecab = MeCab()

def mecab_tokenizer(sentence):
    tokens = mecab.morphs(sentence)
    return tokens

In [ ]:
"""df1["mecab_tokens"] = df1["sentence"].apply(mecab_tokenizer)
df1.to_pickle(base_dir + "datasets/translate.pkl")"""

### Tiktokenizer

In [ ]:
import tiktoken

# 사용할 인코딩 선택
encoding_name = "cl100k_base"  # "p50k_base", "r50k_base", "gpt2" 등으로 변경 가능

# tiktoken 토크나이저 초기화
tokenizer = tiktoken.get_encoding(encoding_name)

def tiktoken_tokenizer(sentence):
    # 문자열이 아닌 경우 빈 시리즈 반환
    if not isinstance(sentence, str):
        return pd.Series([[], []])
    
    # tiktoken을 사용하여 문장을 토큰화하고 ID로 변환
    token_ids = tokenizer.encode(sentence)
    # 토큰 ID를 다시 토큰으로 변환
    token_list = [tokenizer.decode([token_id])[0] for token_id in token_ids]
    return pd.Series([token_ids, token_list])

In [ ]:
df1[["tiktoken_encodes", "tiktoken_tokens"]] = df1["sentence"].apply(tiktoken_tokenizer)
#df2[["tiktoken_encodes", "tiktoken_tokens"]] = df2["sentence"].apply(tiktoken_tokenizer)

### 파일 합쳐서 저장

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df.to_pickle(base_dir + "datasets/total.pkl")

### author 인코딩

In [ ]:
#df = pd.read_pickle(base_dir +  f'data/all.pkl')

In [ ]:
author_set = set(df['Author'].to_list())

author_dict = {author : i for i, author in enumerate(author_set)}

import json

with open(base_dir + "data/author_encoding.json", "w") as json_file:
    json.dump(author_dict, json_file, indent=4)

df['encoded_author'] = df['Author'].apply(lambda x : author_dict[x])

### 장르 인코딩

In [ ]:
#df = pd.read_pickle(base_dir +  f'data/all.pkl')

In [ ]:
df['class'] = df['Classify_1']+"/" + df['Classify_2']

classify_set = set(df['class'].to_list())

classify_dict = {classs : i for i, classs in enumerate(classify_set)}

import json

with open(base_dir + "data/class_encoding.json", "w") as json_file:
    json.dump(classify_dict, json_file, indent=4)

df['encoded_class'] = df['class'].apply(lambda x : classify_dict[x])

In [ ]:
df.to_pickle(base_dir +  f'data/labeled_compact.pkl')

In [ ]:
df